In [2]:
import os
import torch
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

cuda


In [15]:

import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import collections
from datetime import datetime


from pettingzoo.mpe import simple_adversary_v3
from src.utils import rl_tools
from models.maddpg.e2t_maddpg import ENMADDPG
from models.utils import persistence
from src.utils.data_utils import find_latest_file

In [4]:
max_cycles = 200
seed = -1

num_episodes = 100
episode_length = 64 
buffer_size = 100000
hidden_dim = 64
actor_lr = 1e-2
critic_lr = 1e-2
gamma = 0.95
tau = 1e-2
batch_size = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
update_interval = 32
minimal_size = 15
replay_buffer = rl_tools.ReplayBuffer(buffer_size)


cuda


In [14]:
en_out_dim = 768
redu_dim = 16
max_length = 128

In [6]:
max_agents = 6

In [7]:
def add_en_dim(state_dim, redu_dim):

    en_state_dims = []
    for x in state_dim:
        en_state_dims.append(x + redu_dim)
    return en_state_dims

In [8]:
state_dims=[34, 34, 34, 34, 34, 34] 
action_dims=[50, 50, 50, 50, 50, 50] 

en_state_dims = add_en_dim(state_dims, redu_dim)


print(en_state_dims)
print(action_dims)

[50, 50, 50, 50, 50, 50]
[50, 50, 50, 50, 50, 50]


In [9]:
critic_input_dim = sum(state_dims) +sum(action_dims) + redu_dim

In [10]:
print(critic_input_dim)

520


In [12]:
encoders_dir = './parameters/weights/encoders/bert_EN'
spec_en = ''

In [13]:
encoder = torch.load(spec_en) if spec_en else torch.load(find_latest_file(encoders_dir))

In [ ]:
tokenizer = "BertTokenizer"
tokenizer_fn = "tokenizer_fn"

In [39]:
en_maddpg = ENMADDPG(
    tokenizer, 
    tokenizer_fn,
    max_length,
    encoder, 
    en_out_dim, 
    redu_dim, 
    max_agents, 
    device, 
    actor_lr, 
    critic_lr, 
    hidden_dim, 
    state_dims, 
    action_dims, 
    critic_input_dim, 
    gamma, 
    tau
    )
en_maddpg = en_maddpg.to(device)

DDPG 34 50 1272 64 0.01 0.01 cuda
DDPG 34 50 1272 64 0.01 0.01 cuda
DDPG 34 50 1272 64 0.01 0.01 cuda
DDPG 34 50 1272 64 0.01 0.01 cuda
DDPG 34 50 1272 64 0.01 0.01 cuda
DDPG 34 50 1272 64 0.01 0.01 cuda


In [41]:

print(sum(p.numel() for p in en_maddpg.parameters())/1e9,'B parameters in en_maddpg')



0.001144164 B parameters in en_maddpg


In [ ]:
return_list = [] 
total_step = 0


agents = ['adversary_0', 'agent_0', 'agent_1']

In [ ]:
env = simple_adversary_v3.parallel_env(max_cycles=max_cycles)
observations, infos = env.reset() if seed == -1 else env.reset(seed=seed)
print(env.action_spaces)
print(env.observation_spaces)

In [ ]:
critic_input_dim = sum(state_dims) +sum(action_dims)

sa_maddpg = MADDPG(env, device, actor_lr, critic_lr, hidden_dim, state_dims, action_dims, critic_input_dim, gamma, tau)
sa_maddpg = sa_maddpg.to(device)
print(sum(p.numel() for p in sa_maddpg.parameters())/1e6,'M parameters in sa_maddpg')

return_list = [] 
total_step = 0


agents = ['adversary_0', 'agent_0', 'agent_1']
